In [ ]:
import glob
import subprocess
from itertools import count

## Call BigStitcher to stitch overview tiles

In [ ]:

from collections import defaultdict
from pathlib import Path

# make map folder -> number of tiles (need to know for stack-based ImgLaoder)
folders_to_count = defaultdict(int)

for tile in glob.glob('C:/Users/david/Desktop/20230507_imr90_stitching/**/tile_*.tif', recursive=True):
    folders_to_count[Path(tile).parent] += 1

folders_to_count

In [ ]:
# run BigStitcher via Fiji subprocess and macro

script_path = 'C:/Users/david/Desktop/bigstitcher_overview_stitch.ijm'

for root_dir, n_tiles in folders_to_count.items():

    params = f"dir='{root_dir.as_posix()}',n_tiles={n_tiles}"
    fiji_call = f'C:/Users/david/Desktop/Fiji.app/ImageJ-win64.exe --headless --ij2 --run {script_path} {params}'

    res = subprocess.run(fiji_call.split(' '), capture_output=True)
    print(f'{root_dir.as_posix()}: DONE')

## Move stitched overviews to single folder

In [ ]:
import shutil

fused_imgs = glob.glob('/Users/david/Desktop/20230507_imr90_stitching/**/fused.tif', recursive=True)

out_base_path = Path('H:/20230507_imr90_ov_stitch_output')

for fused_img in fused_imgs:
    out_path = out_base_path / (Path(fused_img).parent.name + '.tif')
    shutil.copy(fused_img, out_path)

## Make BigStitcher tileConfic

in tile_configs created by ```overview_extraction_from_h5.ipynb``` replace filename with number (ViewSetupId)

In [ ]:

confs = glob.glob('/Users/david/Desktop/20230507_imr90_stitching/**/tile_config.txt', recursive=True)

for conf in confs:

    ctr = count()
    out_lines = []
    with open(conf) as fd:
        for line in fd.readlines():
            if 'tile' in line:
                _, rest = line.split(';', 1)
                out_lines.append(f'{next(ctr)};{rest}')
            else:
                out_lines.append(line)
    out_lines

    with open(conf.replace('tile_config.txt', 'tile_config_bigstitcher.txt'), 'w') as fd:
        fd.writelines(out_lines)